In [2]:
# %pip install -U langchain langchain-community

In [1]:
from langchain.document_loaders import JSONLoader
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
import json
import os

# Create pull of documents

In [41]:
class RagDataset():
    def __init__(self, filename):
        self.filename = filename
        self.rag_dict = {}
        self.counter = -1

    def __len__(self):
        return len(self.rag_dict)
    
    def get_dict_val(self, prefix, template):
        if isinstance(template, dict):
            for part_template, part_template_val in template.items():
                    result = self.get_dict_val(part_template, part_template_val)
                    if result:
                        text = " ".join([prefix, result])
                        self.rag_dict[self.counter] = {
                            'document': self.filename,
                            'text': text,
                            }


        elif isinstance(template, str):
            if template is None or prefix is None:
                print("None")
            self.counter += 1
            return " ".join([prefix, template])
        else:
            print("error", template)

    def __getitem__(self, idx):
        return self.rag_dict[idx]
    
    def sort(self):
        # sort_dict = dict(sorted(self.rag_dict.items()))
        self.rag_dict = dict(sorted(self.rag_dict.items()))
    

        

In [39]:
for file in os.listdir('./Кодексы'):
    print(file)

ГК_merged.json
ГК_merged_RAG.json
ГПК.json
ГПК_RAG.json
КАС.json
КАС_RAG.json
КоАП.json
КоАП_RAG.json
ТК.json
ТК_RAG.json


In [40]:
codexes = [
    "ГК_merged.json",
    "ГПК.json",
    "КАС.json",
    "КоАП.json",
    "ТК.json"
    ]

In [42]:
for codex in  codexes:
    with open(f"./Кодексы/{codex}", "r", encoding='utf-8') as f:
        parse_codex = json.load(f)
    rag_dataset = RagDataset(codex)
    for razdel, razdel_val in parse_codex.items():
        rag_dataset.get_dict_val(razdel, razdel_val)
    rag_dataset.sort()
    print(f"RAG dataset {codex} created!")
    with open(f"./Кодексы/{codex.split('.')[0]}_RAG.json", "w", encoding='utf-8') as f:
        json.dump(rag_dataset.rag_dict, f)
    print(f"RAG dataset {codex}_RAG.json saved!")
    

RAG dataset ГК_merged.json created!
RAG dataset ГК_merged.json_RAG.json saved!
RAG dataset ГПК.json created!
RAG dataset ГПК.json_RAG.json saved!
RAG dataset КАС.json created!
RAG dataset КАС.json_RAG.json saved!
RAG dataset КоАП.json created!
RAG dataset КоАП.json_RAG.json saved!
RAG dataset ТК.json created!
RAG dataset ТК.json_RAG.json saved!


In [51]:
RAG_DOCS = []
for file in os.listdir("./Кодексы"):
    if "_RAG" in file:
        with open(f"./Кодексы/{file}", "r",  encoding='utf-8') as f:
            rag_doc = json.load(f)

        print(f"RAG doc {file} append!")

        RAG_DOCS.append(rag_doc)

RAG doc ГК_merged_RAG.json append!
RAG doc ГПК_RAG.json append!
RAG doc КАС_RAG.json append!
RAG doc КоАП_RAG.json append!
RAG doc ТК_RAG.json append!


In [52]:
RAG_TEXTS = []
for period, codex in enumerate(RAG_DOCS):
    for idx, (_, text) in enumerate(codex.items()):
        RAG_TEXTS.append(text)
len(RAG_TEXTS)

4257

In [56]:
RAG_TEXTS[:5]

[{'document': 'ГК_merged.json',
  'text': 'Глава 1. Гражданское законодательство Статья 1. Основные начала гражданского законодательства 1. Гражданское законодательство основывается на признании равенства участников регулируемых им отношений, неприкосновенности собственности, свободы договора, недопустимости произвольного вмешательства кого-либо в частные дела, необходимости беспрепятственного осуществления гражданских прав, обеспечения восстановления нарушенных прав, их судебной защиты.2. Граждане (физические лица) и юридические лица приобретают и осуществляют свои гражданские права своей волей и в своем интересе. Они свободны в установлении своих прав и обязанностей на основе договора и в определении любых не противоречащих законодательству условий договора.Гражданские права могут быть ограничены на основании федерального закона и только в той мере, в какой это необходимо в целях защиты основ конституционного строя, нравственности, здоровья, прав и законных интересов других лиц, обес

In [57]:
documents = [Document(page_content=text['text'], metadata={"document": text['document']}) for text in RAG_TEXTS]
documents[:5]

[Document(metadata={'document': 'ГК_merged.json'}, page_content='Глава 1. Гражданское законодательство Статья 1. Основные начала гражданского законодательства 1. Гражданское законодательство основывается на признании равенства участников регулируемых им отношений, неприкосновенности собственности, свободы договора, недопустимости произвольного вмешательства кого-либо в частные дела, необходимости беспрепятственного осуществления гражданских прав, обеспечения восстановления нарушенных прав, их судебной защиты.2. Граждане (физические лица) и юридические лица приобретают и осуществляют свои гражданские права своей волей и в своем интересе. Они свободны в установлении своих прав и обязанностей на основе договора и в определении любых не противоречащих законодательству условий договора.Гражданские права могут быть ограничены на основании федерального закона и только в той мере, в какой это необходимо в целях защиты основ конституционного строя, нравственности, здоровья, прав и законных инте

In [59]:
import numpy as np

In [61]:
lens = np.asarray([len(x['text'].split()) for x in RAG_TEXTS])

In [62]:
lens.max(), lens.min(), lens.mean()

(np.int64(8192), np.int64(15), np.float64(185.19262391355414))

In [63]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [71]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

In [72]:
splits = text_splitter.split_documents(documents)
splits[:5]

[Document(metadata={'document': 'ГК_merged.json'}, page_content='Глава 1. Гражданское законодательство Статья 1. Основные начала гражданского законодательства 1. Гражданское законодательство основывается на признании равенства участников регулируемых им отношений, неприкосновенности собственности, свободы договора, недопустимости произвольного вмешательства кого-либо в частные дела, необходимости беспрепятственного осуществления гражданских прав, обеспечения восстановления нарушенных прав, их судебной защиты.2. Граждане (физические лица) и юридические лица приобретают и осуществляют свои гражданские права своей волей и в своем интересе. Они свободны в установлении своих прав и обязанностей на основе договора и в определении любых не противоречащих законодательству условий договора.Гражданские права могут быть ограничены на основании федерального закона и только в той мере, в какой это необходимо в целях защиты основ конституционного строя, нравственности, здоровья, прав и законных инте

In [73]:
lens = np.asarray([len(x.page_content.split()) for x in splits])

In [74]:
lens.max(), lens.min(), lens.mean(), len(lens)

(np.int64(364), np.int64(15), np.float64(143.77519650655023), 5725)

In [2]:
embedder = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

C:\Users\rtyde\AppData\Local\Temp\ipykernel_7656\135043871.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")


In [81]:
vector_store = FAISS.from_documents(splits, embedder)
vector_store.save_local("faiss_legal_index")

In [4]:
vector_store = FAISS.load_local("faiss_legal_index", embedder, allow_dangerous_deserialization=True)

In [36]:
retriever = vector_store.as_retriever(search_kwargs={"k":5,
                                                    #   "filter":{"metadata":"document"}
                                                     })

In [37]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025880776C10>, search_kwargs={'k': 5})

In [38]:
retriever.invoke(
    "Гражданское право даёт человеку"
)

[Document(id='15bdb498-0c2c-4863-b732-650577eab236', metadata={'document': 'КоАП.json'}, page_content='Глава 12. Административные правонарушения в области дорожного движения Статья 12.17.1. Утратила силу . - Федеральный закон от 21.07.2014 N 210-ФЗ.'),
 Document(id='61a6828d-cfb7-4b31-ab52-345f32010408', metadata={'document': 'КоАП.json'}, page_content='Глава 13. Административные правонарушения в области связи и информации Статья 13.30. Утратила силу  с 1 марта 2023 года. - Федеральный закон от 29.12.2022 N 625-ФЗ.'),
 Document(id='abe5296c-3e55-414d-8af7-6cfb571f21a1', metadata={'document': 'ТК.json'}, page_content='Раздел XIV. Заключительные положения Статья 422. Признание утратившими силу отдельных законодательных актов Признать утратившими силу с 1 февраля 2002 года:Кодекс законов о труде РСФСР, утвержденный Законом РСФСР от 9 декабря 1971 года "Об утверждении Кодекса законов о труде РСФСР" (Ведомости Верховного Совета РСФСР, 1971, N 50, ст. 1007);Указ Президиума Верховного Совета 